In [40]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [41]:
class AITextDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

In [47]:
data = pd.read_csv("feature_engineered_embeddings")
shuffled = data.sample(frac = 1)
splits = np.array_split(shuffled, 2)

/Users/devammondal/PycharmProjects/lstm/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [52]:
print(torch.tensor(float(data['embeddings'][0])))

ValueError: could not convert string to float: '[[-0.0809724  -0.08486272 -0.6065865  ... -0.4288915   0.49864244\n   0.21013789]\n [ 0.16275167  0.19716358 -0.63289344 ...  0.6522909   1.16892\n   0.40316024]\n [ 0.0872129   0.29124722  0.32916912 ...  0.7439031   1.0377266\n   0.703288  ]\n ...\n [ 0.20495985  0.5605411   0.2448235  ... -0.22918493 -0.01759733\n  -0.2825431 ]\n [ 0.08957516 -0.01565889  0.3656449  ... -0.08716358 -0.15552682\n  -0.48998094]\n [ 0.17319556  0.28364557  0.566096   ... -0.13319233 -0.12998258\n  -0.47630036]]'

In [32]:
train = splits[0]
test = splits[1]

In [33]:
print(train)

                                                  text  generated  \
383  There are many advantages to limiting car usag...          0   
291  The Electoral College is a process that the fo...          0   
198  I think asking multiple people for advice is a...          0   
275  Do i think the technology is a good value . Ye...          0   
136  The use of technology that is able to read the...          0   
..                                                 ...        ...   
464  Sticky situations, we've all been through one ...          0   
284  \nTechnology has profoundly impacted humanity ...          1   
415  Phones and Driving\n\nAs of 2017, an estimated...          0   
97   Space exploration has been a topic of debate s...          1   
482  The Face on Mars is a popular pop icon dating ...          0   

                                            embeddings  
383  tensor([[-0.1165, -0.3161, -0.2490,  ..., -0.1...  
291  tensor([[-0.2774, -0.0515,  0.1036,  ..., -0.1...  


In [34]:
train_dataset = AITextDataset(data=list(train.embeddings), labels=list(train.generated))
test_dataset = AITextDataset(data=list(test.embeddings), labels=list(train.generated))

In [35]:
class LSTM(torch.nn.Module):
    def __init__(self, input, hidden, layers, output):
        super(LSTM, self).__init__()
        self.hidden = hidden
        self.layers = layers
        self.lstm = torch.nn.LSTM(input, hidden, layers, output, batch_first = True)
        self.connected = torch.nn.Linear(hidden, output)
        
    def forward(self, x):
        hidden_state = torch.zeros(self.layers, x.size(0), self.hidden)
        memory = torch.zeros(self.layers, x.size(0), self.hidden)
        out, _  = self.lstm(x, (hidden_state, memory))
        out = self.fc(out[:, -1, :])
        return out

In [36]:
model = LSTM(768, 64, 2, 2)

In [37]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters())

In [38]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle = True)

In [39]:
EPOCHS = 100
train_loss = []
train_acc = []
test_acc = []

for epoch in range(EPOCHS):
    for i, data in enumerate(train_loader):
        tensor, label = data
        tensor = torch.tensor(tensor[0])
        
        prediction = model(tensor[0])
        
        loss = loss_function(prediction, label)
        accuracy = (prediction.round() == label).float().mean()
        
        train_loss.append(loss)
        train_acc.append(accuracy)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        print(f"epoch {epoch} step {i} loss {loss} accuracy {accuracy}")

TypeError: new(): invalid data type 'str'